# Setup

In [1]:
from __future__ import print_function
import torch
print(torch.__version__)
import torchvision
print(torchvision.__version__)

import numpy as np
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

1.5.0
0.6.0
CUDA is not available.  Training on CPU ...


In [2]:
import torchvision.datasets as dset
from torchvision import  transforms
import collections

# path to your own data and coco file
train_data_dir = '/data/cmpe256-01-su2020/coco/train2017'
val_data_dir = '/data/cmpe256-01-su2020/coco/val2017'
train_coco = '/data/cmpe256-01-su2020/coco/annotations/instances_train2017.json'
val_coco = '/data/cmpe256-01-su2020/coco/annotations/instances_val2017.json'
transform = transforms.Compose([transforms.Resize((480, 640)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,)),
                               ])

coco_val = dset.CocoDetection(root = val_data_dir,
                                annFile = val_coco,
                                transform=transform)

coco_train = dset.CocoDetection(root = train_data_dir,
                                annFile = train_coco,
                                transform=transform)

train_loader = torch.utils.data.DataLoader(coco_train, batch_size=12)
val_loader = torch.utils.data.DataLoader(coco_train, batch_size=12)

loading annotations into memory...
Done (t=0.52s)
creating index...
index created!
loading annotations into memory...
Done (t=14.00s)
creating index...
index created!


In [3]:
print('Number of samples: ', len(coco_train))

Number of samples:  118287


In [4]:
from torchvision import models
import torch.nn as nn

criterion = nn.NLLLoss()

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# our dataset has two classes only - raccoon and not racoon
num_classes = 2
# get the model using our helper function
model = models.mobilenet_v2(pretrained=False)
# move model to the right device
model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [11]:
import numpy as np

# number of epochs to train the model
n_epochs = 50

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train() # prep model for training
    for data, target in train_loader: #one batch data
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
#         print (output.size())
#         print (output)
        for x in target[0]:
            print (x)
            print (target[0][x])
#         print(target[0]['segmentation'][0])
#         print (target[0]['bbox'])
#         print (torch.stack(target[0]['bbox']).size())
        loss = criterion(output, target[0]['area'])
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)#batch size
        
    ######################    
    # validate the model #
    ######################
    model.eval() # prep model for evaluation
    for data, target in valid_loader:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update running validation loss 
        valid_loss += loss.item()*data.size(0)
        
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)#calculate average loss
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        valid_loss
        ))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model.pt')
        valid_loss_min = valid_loss

segmentation
[[tensor([500.4900, 437.5200, 267.3800,   4.7800, 164.5000, 382.4800, 181.5900,
        261.2000,  52.3600, 468.7300, 166.8300, 134.3600], dtype=torch.float64), tensor([473.5300, 353.3300, 330.1400,  22.9200, 479.3800, 268.6300, 363.4300,
        251.7100, 387.7700, 228.3600, 509.1200, 145.5500], dtype=torch.float64), tensor([599.7300, 437.8700, 281.8100,  75.4500, 120.2600, 330.2400, 186.8000,
        270.8200, 100.6800, 468.7300, 240.1800, 117.0200], dtype=torch.float64), tensor([419.6000, 326.9800, 314.7500,  39.1600, 448.4000, 229.9300, 362.1300,
        259.0400, 391.4800, 224.7300, 445.8400, 155.6700], dtype=torch.float64), tensor([612.6700, 433.6500, 299.1200, 148.0300,  93.7000, 278.9700, 186.8000,
        274.9400, 123.8100, 473.2600, 256.0000, 115.5800], dtype=torch.float64), tensor([375.3700, 306.2600, 282.0500,  46.8000, 442.8700, 205.7500, 354.9700,
        259.0400, 395.2000, 222.5500, 418.5200, 168.6700], dtype=torch.float64), tensor([608.3600, 427.3300, 281

RuntimeError: expected scalar type Long but found Double